# Aufgabe - Impfdaten
Kurz nach dem Bekanntwerden des Virus haben die ersten Länder auf Erde-14 mit der Impfstoffentwicklung begonnen, da allen klar war, dass das Virus nur mit einer Herdenimmunität, beispielsweise einer Impfung, kontrollieren werden kann. Ende des Jahres 2052 waren die ersten Impfstoffe bereit für die Bevölkerung. Die ersten drei zugelassenen Impfstoffe sind von den Unternehmen Bioneca, Moderntech und Asterna. Da das Land Dendenland eine ähnliche Gesellschaft zu Deutschland aufweist, werden Sie zusätzlich beauftragt den Impfablauf auszuwerten, um aus möglichen Fehlern zu lernen. Dazu wird Ihnen der folgende Datensatz bereitgestellt:

In [ ]:
%use krangl
val datenRaw = DataFrame.readCSV("https://raw.githubusercontent.com/Limatationz/kotlin-mooc/main/Abschnitt2/daten/impfzahlenDendenland.csv") //Liest die csv-Datei und wandelt sie in einen DataFrame um
val daten = datenRaw.filterByRow { 
    (!(it["metric"]as String).contains("erst") && !(it["metric"]as String).contains("voll") && !(it["metric"]as String).contains("impfstelle")) || it["metric"] == "personen_erst_kumulativ" || it["metric"] == "personen_voll_kumulativ"
}

Der Aufbau der Daten ist folgendermaßen:

In [ ]:
daten.schema()

In [ ]:
daten.head(20)

## Die Daten kennenlernen
In dieser Teilaufgabe sollen die Daten und deren Struktur kennengelernt werden. Es ist zu erkennen, dass in der Spalte *metric* eine Liste an Beschreibungen für die Daten, die in *value* zu finden sein, vorliegt. Implementieren Sie eine Erweiterungsmethode `gibBeschreibungen()` der Klasse `DataFrame`, die alle Beschreibungen in einer Liste zurückgibt.  
Beispiel: [dosen_kumulativ, personen_erst_kumulativ, personen_voll_kumulativ, ...]

In [ ]:
fun DataFrame.gibBeschreibungen(): List<String>{
    // YOUR CODE HERE
    throw NotImplementedError()
}

In [ ]:
val ergebnis = listOf("dosen_kumulativ", "personen_erst_kumulativ", "personen_voll_kumulativ", "dosen_bioneca_kumulativ", "dosen_moderntech_kumulativ", "dosen_asterna_kumulativ", "indikation_alter_dosen", "indikation_beruf_dosen", "indikation_medizinisch_dosen", "indikation_pflegeheim_dosen", "dosen_differenz_zum_vortag", "impf_quote_dosen", "impf_inzidenz_dosen", "dosen_kumulativ_alter_unter60", "dosen_kumulativ_alter_60plus")
if(daten.gibBeschreibungen() != ergebnis)
    throw AssertionError("daten.gibBeschreibungen(): ${daten.gibBeschreibungen()} != ${ergebnis}")

## Wieviele Dosen wurden an einem bestimmten Tag verimpft?
Implementieren Sie eine Erweiterungsmethode `gibAnzahlDosen(datum, region)` der Klasse `DataFrame`, die die Anzahl der Impfdosen, welche an dem übergebenen Datum und in der übergebenen Region verimpft wurden, zurückgibt. Zur Vereinfachung ist anzunehmen, dass das Datum als `String` im Format `yyyy-mm-dd` und die Region als `String` mit zwei Buchstaben übergeben werden. In dieser Aufgabe soll nicht mit dem Eintrag `dosen_differenz_zum_vortag` gearbeitet werden, da die Daten unvollständig sind (näheres dazu in der nächsten Aufgabe). Ist das Datum nicht in den Daten vorhanden, soll `-1` zurückgegeben werden.

In [ ]:
fun DataFrame.gibAnzahlDosen(datum: String, region: String): Int {
    // YOUR CODE HERE
    throw NotImplementedError()
}

In [ ]:
val tests = mapOf("2053-04-09" to 544898, "2052-12-27" to 21566, "2053-01-18" to 74406, "2054-03-05" to -1)
for ((datum, anzahl) in tests){
    if (daten.gibAnzahlDosen(datum, "DE") != anzahl)
        throw AssertionError("daten.gibAnzahlDosen($datum): ${daten.gibAnzahlDosen(datum, "DE")} != ${anzahl}")
}

## Verbesserung der Daten I
Mit der Methode `gibAnzahlDosen(datum, region)` kann nun die Differenz der Dosen zum Vortag ausgerechnet werden. Da der Datensatz einige Lücken bezüglich des Eintrags `dosen_differenz_zum_vortag` aufweist, sollen diese jetzt vefüllt werden. Dazu bauen wir uns einen neuen Datensatz `datenVollstaendig` in der Funktion `vervollstaenigeLuecken(daten)` zusammen. Das Grundgerüst des `DataFrame`s ist bereits vorbereitet. Eine Zeile kann mit der Methode [`bindRows(map)`](https://holgerbrandl.github.io/krangl/javadoc/krangl/krangl/bind-rows.html) angehängt werden.

In [ ]:
fun vervollstaendigeLuecken(daten: DataFrame): DataFrame {
    var datenVollstaendig: DataFrame = emptyDataFrame()
    // YOUR CODE HERE
    throw NotImplementedError()
    return datenVollstaendig
}


In [ ]:
val testdaten = vervollstaendigeLuecken(daten)
val luecken = (testdaten.filterByRow { it["metric"] == "dosen_differenz_zum_vortag" && it["value"] == "" }.count().get(0).get(0) as Int)
if (luecken > 0)
    throw AssertionError("Anzahl der Lücken: ${luecken} > 0")

## Verbesserung der Daten II
Bei genaueren Betrachtung der Daten mit einem genaueren Blick auf die Datentypen der Spalten, fällt auf, dass alle Werte der Spalte *value* den Datentyp `String` besitzen. Da aber in dieser Spalte nur Fließkommazahlen oder nichts vorzufinden ist, würde der Datenyp `Double` besser passen. Passen Sie Ihre Lösung von *Verbesserung der Daten I* insoweit an, dass in der Spalte *value* nur noch Werte des Datentyps `Double` zu finden sind, falls ein gültiger Wert enthalten ist. Ansonsten soll `null` eingefügt werden. Schreiben Sie die Lösung zu dieser Aufgabe in die Funktion `vervollstaenigeDaten(daten)`.

In [ ]:
fun vervollstaendigeDaten(daten: DataFrame): DataFrame {
    var datenVollstaendig: DataFrame = emptyDataFrame()
    // YOUR CODE HERE
    throw NotImplementedError()
    return datenVollstaendig
}

In [ ]:
val testdaten = vervollstaendigeDaten(daten)
val luecken = (testdaten.filterByRow { it["metric"] == "dosen_differenz_zum_vortag" && it["value"] == null}.count().get(0).get(0) as Int)
if (luecken > 0)
    throw AssertionError("Anzahl der Lücken: ${luecken} > 0")
if (testdaten.get("value")::class.simpleName != "DoubleCol")
    throw AssertionError("Datentyp der Spalte value: ${testdaten.get("value")::class.simpleName} != DoubleCol")

## Wann wurden am meisten Dosen mit der Indikaton Alter verimpft?
In den Daten ist außerdem angegeben aus welchem Grund eine Impfung verabreicht wurde. Implementieren Sie eine Erweiterungsmethode `gibDatumAlter()`, die das Datum des Tages zurückgibt, an dem am meisten Dosen augrund der Indikation Alter verabreicht wurden.

In [ ]:
fun DataFrame.gibDatumAlter(): String {
    // YOUR CODE HERE
    throw NotImplementedError()
}

In [ ]:
val testdaten = vervollstaendigeDaten(daten)
if (testdaten.gibDatumAlter() != "2053-04-05")
    throw AssertionError("testdaten.gibDatumAlter(): ${testdaten.gibDatumAlter()} != 2053-04-05")

## Wer wird eigentlich geimpft?
In dieser Aufgabe soll der Frage nachgegangen werden, wer eigentlich geimpft wird. In den Daten gibt es dazu 4 Metriken: *indikation_alter_dosen*, *indikation_beruf_dosen*, *indikation_medizinisch_dosen*, *indikation_pflegeheim_dosen*. Implementieren Sie eine Erweiterungsmethode `gibIndikationen()` der Klasse `DataFrame`, die eine Liste des Datentyps `List<Map<String, Map<String, Any>>>` zurückgibt. In dieser soll für jedes Datum eine Map, in der die verschiedenenen Indikationen der Schlüssel (Alter, Beruf, Medizinisch, Pflegeheim) sind, existieren. Die Werte zu den Indikationen sind die an diesem Tag verimpften Dosen der Infikation. Außerdem soll unter dem Schlüssel *Max* eine weitere Map zu finden sein, die als Schlüssel die Indikation mit den meisten Dosen an diesem Datum und als Wert die Anzahl besitzt.  
Beispiel:  
[{2052-12-27={Alter=5833.0, Beruf=7618.0, Medizinisch=962.0, Pflegeheim=11646.0, Max={Pflegeheim=11646.0}}}, {2052-12-28={Alter=6399.0, Beruf=7346.0, Medizinisch=699.0, Pflegeheim=10121.0, Max={Pflegeheim=10121.0}}}, ... ]

In [ ]:
fun DataFrame.gibIndikationen(): List<Map<String, Map<String, Any>>> {
    // YOUR CODE HERE
    throw NotImplementedError()
}

In [ ]:
val testdaten = vervollstaendigeDaten(daten).gibIndikationen()
val testZeilen = listOf("{2052-12-27={Alter=5833.0, Beruf=7618.0, Medizinisch=962.0, Pflegeheim=11646.0, Max={Pflegeheim=11646.0}}}", "{2052-12-28={Alter=6399.0, Beruf=7346.0, Medizinisch=699.0, Pflegeheim=10121.0, Max={Pflegeheim=10121.0}}}")
for (i in 0 until testZeilen.size){
    if (testdaten[i].toString() != testZeilen[i])
        throw AssertionError("${testdaten[i].toString()} != ${testZeilen[i]}")
}

testdaten.forEach {
    val max = it.values.toList()[0].values.take(4).toList().maxByOrNull { it as Double }
    var string = it.values.toList()[0].values.drop(4)[0].toString()
    val (indikator, dosen) = string.substring(1, string.length-1).split("=")
    if(max != dosen.toDouble())
        throw AssertionError("Maximum in Zeile $it: $dosen != $max")
}